[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Training_Tutorial.ipynb)

## Create the models
Import the models from the ISR package and create

- a RRDN super scaling network
- a discriminator network for GANs training
- a VGG19 feature extractor to train with a perceptual loss function

Carefully select
- 'x': this is the upscaling factor (2 by default)
- 'layers_to_extract': these are the layers from the VGG19 that will be used in the perceptual loss (leave the default if you're not familiar with it)
- 'lr_patch_size': this is the size of the patches that will be extracted from the LR images and fed to the ISR network during training time

Play around with the other architecture parameters

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
# import tensorflow as tf
# # 查看gpu和cpu的数量
# gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
# cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
# print("Num GPUs Available: ", len((tf.config.experimental.list_physical_devices('GPU'))))
# print(gpus, cpus)

In [3]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

In [4]:

# import torch
# print(torch.__version__)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# #返回cuda表示成功
# #或者
# print(torch.cuda.is_available())
# #返回True表示成功

In [5]:
print(os.getcwd())

d:\Master_Thesis\image-super-resolution-master\notebooks


In [6]:
lr_train_patch_size = 40#40
layers_to_extract = [5, 9]
scale = 2
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

In [7]:
print(os.getcwd())

d:\Master_Thesis\image-super-resolution-master\notebooks


## Give the models to the Trainer
The Trainer object will combine the networks, manage your training data and keep you up-to-date with the training progress through Tensorboard and the command line.

Here we do not use  the pixel-wise MSE but only the perceptual loss by specifying the respective weights in `loss_weights`

In [8]:
from ISR.train import Trainer
loss_weights = {
  'generator': 0.0,
  'feature_extractor': 0.0833,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

path = os.getcwd()#d:/Master_Thesis/image-super-resolution-master/notebooks
log_dirs = {'logs': path + '/ISR_chr/logs', 'weights': path + '/ISR_chr/weights'}

learning_rate = {'initial_value': 0.001, 'decay_factor': 0.5, 'decay_frequency': 30}
#learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}
flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    # lr_train_dir=path + '/ISR_chr/data/train_LR/',
    # hr_train_dir=path + '/ISR_chr/data/train_HR_24/',
    # lr_valid_dir=path + '/ISR_chr/data/valid_LR/',
    # hr_valid_dir=path + '/ISR_chr/data/valid_HR_2/',
    # lr_train_dir=path + '/ISR_chr/data/final_data/data_aug/train/',
    # hr_train_dir=path + '/ISR_chr/data/final_data/labels_aug/train/',
    # lr_valid_dir=path + '/ISR_chr/data/final_data/data_aug/val/',
    #hr_valid_dir=path + '/ISR_chr/data/final_data/labels_aug/val/',

    # lr_train_dir=path + '/ISR_chr/data/100_70/final2/bw70/train/',
    # lr_valid_dir=path + '/ISR_chr/data/100_70/final2/bw70/val/',

    # hr_train_dir=path + '/ISR_chr/data/100_70/final2/bw100_x2/train/',
    # hr_valid_dir=path + '/ISR_chr/data/100_70/final2/bw100_x2/val/',
    lr_train_dir=path + '/ISR_chr/data/all_data/SoS_ISR/data/train/',
    lr_valid_dir=path + '/ISR_chr/data/all_data/SoS_ISR/data/val/',

    hr_train_dir=path + '/ISR_chr/data/all_data/SoS_ISR/label/train/',
    hr_valid_dir=path + '/ISR_chr/data/all_data/SoS_ISR/label/val/',

    loss_weights=loss_weights,
    learning_rate=learning_rate,
    flatness=flatness,
    dataname='div2k',
    log_dirs=log_dirs,
    weights_generator=None,
    weights_discriminator=None,
    n_validation=200,#33,#40
)


Choose epoch number, steps and batch size and start training

In [9]:
trainer.train(
    epochs=300,#10
    steps_per_epoch=20,#20
    batch_size=4,#4
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)


Training details:
  training_parameters: 
    lr_train_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/all_data/SoS_ISR/data/train/
    hr_train_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/all_data/SoS_ISR/label/train/
    lr_valid_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/all_data/SoS_ISR/data/val/
    hr_valid_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/all_data/SoS_ISR/label/val/
    loss_weights: {'generator': 0.0, 'feature_extractor': 0.0833, 'discriminator': 0.01}
    log_dirs: {'logs': 'd:\\Master_Thesis\\image-super-resolution-master\\notebooks/ISR_chr/logs', 'weights': 'd:\\Master_Thesis\\image-super-resolution-master\\notebooks/ISR_chr/weights'}
    fallback_save_every_n_epochs: 2
    dataname: div2k
    n_validation: 200
    flatness: {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}
    learning_rate: {'initial_value': 0.001, 'decay_f

Epoch 0/300
Current learning rate: 0.0010000000474974513
100%|██████████| 20/20 [03:27<00:00, 10.38s/it]
Epoch 0 took      207.5s


800/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.3131336671859026, 'val_generator_loss': 0.4706031, 'val_discriminator_loss': 0.006239352, 'val_feature_extractor_loss': 1.2939923, 'val_feature_extractor_1_loss': 6.2227244, 'val_generator_PSNR_Y': 7.283228, 'train_d_real_loss': 0.021746207, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 0.021806624, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.41767597, 'train_generator_loss': 0.5362718, 'train_discriminator_loss': 0.0063867797, 'train_feature_extractor_loss': 1.5767401, 'train_feature_extractor_1_loss': 8.449961, 'train_generator_PSNR_Y': 5.8682528}
val_generator_PSNR_Y improved from       -inf to    7.28323
Saving weights
Epoch 1/300
Current learning rate: 0.0010000000474974513
100%|██████████| 20/20 [03:36<00:00, 10.84s/it]
Epoch 1 took      216.7s


800/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

...

{'val_loss': 0.07471951224841178, 'val_generator_loss': 0.1740314, 'val_discriminator_loss': 0.42297584, 'val_feature_extractor_loss': 0.327889, 'val_feature_extractor_1_loss': 1.3645422, 'val_generator_PSNR_Y': 20.836973, 'train_d_real_loss': 1.4088182e-05, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 1.3177965e-05, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.056484662, 'train_generator_loss': 0.20247914, 'train_discriminator_loss': 0.513609, 'train_feature_extractor_loss': 0.2573862, 'train_feature_extractor_1_loss': 0.9754726, 'train_generator_PSNR_Y': 18.691435}
val_generator_PSNR_Y did not improve.
Saving weights anyways.
Epoch 105/300
Current learning rate: 0.0001250000059371814
Current flatness treshold: 0.15
100%|██████████| 20/20 [03:30<00:00, 10.51s/it]
Epoch 105 took      210.3s


800/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.07530232666991651, 'val_generator_loss': 0.1721139, 'val_discriminator_loss': 0.5399635, 'val_feature_extractor_loss': 0.333003, 'val_feature_extractor_1_loss': 1.3453326, 'val_generator_PSNR_Y': 20.661732, 'train_d_real_loss': 1.6423675e-05, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 1.542216e-05, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.09163352, 'train_generator_loss': 0.1905942, 'train_discriminator_loss': 0.62808156, 'train_feature_extractor_loss': 0.47366142, 'train_feature_extractor_1_loss': 1.5756232, 'train_generator_PSNR_Y': 19.204407}
val_generator_PSNR_Y did not improve.
Epoch 106/300
Current learning rate: 0.0001250000059371814
Current flatness treshold: 0.15
100%|██████████| 20/20 [03:30<00:00, 10.53s/it]
Epoch 106 took      210.6s


104/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
#%tensorboard --logdirs= 'D:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/logs/rrdn-C4-D3-G64-G064-T10-x2/2024-03-15_1048/'
%tensorboard --logdirs= 'D:/Master_Thesis/image-super-resolution-master/notebooks/ISR_chr/logs/rrdn-C4-D3-G64-G064-T10-x2/2024-03-15_1048/'

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--purge_orphaned_data BOOL] [--db URI] [--db_import]
                   [--inspect] [--version_tb] [--tag TAG] [--event_file PATH]
                   [--path_prefix PATH] [--window_title TEXT]
                   [--max_reload_threads COUNT] [--reload_interval SECONDS]
                   [--reload_task TYPE] [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [--debugger_data_server_grpc_port PORT]
                   [--debugger_port PORT] [--master_tpu_unsecure_channel ADDR]
                   [--whatif-use-unsafe-custom-prediction YOUR_CUSTOM_PREDICT_FUNCTION.py]
                   [--whatif-data-dir PA

In [ ]:
%load_ext tensorboard